In [1]:
from TimeLLMProcessors import FixedModelTimeLLMProcessor, UpdatingModelTimeLLMProcessor

In [2]:
import pandas as pd

In [3]:
updated_df = pd.read_csv("https://raw.githubusercontent.com/cdcepi/FluSight-forecast-hub/refs/heads/main/target-data/target-hospital-admissions.csv")
updated_df = updated_df[updated_df["location_name"] == "US"]
updated_df = updated_df[["date", "value"]]
updated_df["date"] = pd.to_datetime(updated_df["date"])
updated_df.set_index("date", inplace = True)
updated_df.sort_values(by = "date", inplace = True)

In [4]:
Processor = FixedModelTimeLLMProcessor(overall_df = updated_df, dates = ["2024-10-05", "2024-10-19", "2024-11-02", "2024-11-16", "2024-12-07"])

In [5]:
Processor.create_training_dfs(value_col = "value")

In [6]:
prompt = "This dataset contains data on weekly flu hospitalizations in the United States. There is a yearly (52 weeks) seasonality"

config = {
    "input_size": 8,
    "windows_batch_size": {"type": "categorical",
                           "values": [32, 64, 128]},
    "inference_windows_batch_size": {"type": "integer",
                                     "values": [32, 128]}

}

In [11]:
Processor.create_fixed_model(h = 4, freq = "W-SAT", prompt = prompt, config = config, model_name = "OptunaTest", window_size = 30)

[I 2025-06-23 14:31:50,973] A new study created in memory with name: no-name-c19c0707-1088-465a-8420-ae6bf7d56083
[rank: 0] Seed set to 1


Successfully loaded model: gpt2-medium


/home/hmf6av/.local/lib/python3.11/site-packages/neuralforecast/common/_base_model.py:537: UserWarning: val_check_steps is greater than max_steps, setting val_check_steps to max_steps.
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type               | Params | Mode 
-------------------------------------------------------------------
0 | loss                | MAE                | 0      | train
1 | padder_train        | ConstantPad1d      | 0      | train
2 | scaler              | TemporalNorm       | 0      | train
3 | llm                 | GPT2Model          | 354 M  | eval 
4 | patch_embedding     | PatchEmbedding     | 1.5 K  | train
5 | mapping_layer       | Linear             | 51.5 M | train
6 | reprogramming_layer | ReprogrammingLayer | 3.2 M  | train
7 | output_projection   | FlattenHead        | 516    | train
8 | normalize

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

[W 2025-06-23 14:31:53,903] Trial 0 failed with parameters: {'windows_batch_size': 64, 'inference_windows_batch_size': 44} because of the following error: OutOfMemoryError('CUDA out of memory. Tried to allocate 126.00 MiB. GPU 0 has a total capacity of 31.73 GiB of which 120.38 MiB is free. Including non-PyTorch memory, this process has 31.40 GiB memory in use. Of the allocated memory 30.85 GiB is allocated by PyTorch, and 162.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)').
Traceback (most recent call last):
  File "/home/hmf6av/.local/lib/python3.11/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/sfs/gpfs/tardis/home/hmf6av/TimeSeriesComparisons/TimeL

OutOfMemoryError: CUDA out of memory. Tried to allocate 126.00 MiB. GPU 0 has a total capacity of 31.73 GiB of which 120.38 MiB is free. Including non-PyTorch memory, this process has 31.40 GiB memory in use. Of the allocated memory 30.85 GiB is allocated by PyTorch, and 162.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from neuralforecast.models import TimeLLM

In [ ]:
timellm = TimeLLM(h = 4, prompt_prefix = prompt, input_size = 10, llm = 'gpt2-medium', d_llm = 1024)

In [ ]:
from transformers import AutoConfig, AutoModelForCausalLM

In [ ]:
# 1. Load the _full_ pretrained model (with its 1,024 positions)
model = AutoModelForCausalLM.from_pretrained("gpt2-medium")

# 2. Now shrink its position embeddings down to 16 slots
model.resize_token_embeddings(16)


In [ ]:
model.get_input_embeddings()

In [ ]:
timellm.llm.get_input_embeddings()

In [ ]:
model.config

In [ ]:
timellm.llm.config

In [ ]:
model?

In [4]:
UProcessor = UpdatingModelTimeLLMProcessor(overall_df=updated_df, dates=["2024-10-05", "2024-10-19", "2024-11-02", "2024-11-16", "2024-12-07"])

In [5]:
UProcessor.create_training_dfs(value_col="value")

/sfs/gpfs/tardis/home/hmf6av/TimeSeriesComparisons/TimeLLMProcessors.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ds'] = df.index
/sfs/gpfs/tardis/home/hmf6av/TimeSeriesComparisons/TimeLLMProcessors.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["unique_id"] = "series_1"
/sfs/gpfs/tardis/home/hmf6av/TimeSeriesComparisons/TimeLLMProcessors.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [9]:
UProcessor.create_models(h = 4, freq = "W-SAT", prompt = prompt, config = config, model_names = ["test1", "test2", "test3", "test4", "test5"], window_size = 30)

[I 2025-06-23 16:13:58,961] A new study created in memory with name: no-name-a570e408-df73-4096-ab8d-f331c3a5503f
[rank: 0] Seed set to 1
2025-06-23 16:14:05.924569: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-23 16:14:06.995187: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-23 16:14:08.453850: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-23 16:14:08.453900: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-2

Successfully loaded model: gpt2-medium


/home/hmf6av/.local/lib/python3.11/site-packages/neuralforecast/common/_base_model.py:537: UserWarning: val_check_steps is greater than max_steps, setting val_check_steps to max_steps.
  warnings.warn(
/home/hmf6av/.local/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /apps/software/standard/core/jupyterlab/3.6.3-py3.11 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type               | Params | Mode 
-------------------------------------------------------------------
0 | loss                | MAE                | 0      | train
1 | padder_train        | ConstantPad1d      | 0      | train
2 | scaler              | TemporalNorm       | 0

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/hmf6av/.local/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=35` in the `DataLoader` to improve performance.
/home/hmf6av/.local/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=35` in the `DataLoader` to improve performance.
/home/hmf6av/.local/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/hmf6av/.local/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=35` in the `DataLoader` to improve performance.
/home/hmf6av/.local/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=35` in the `DataLoader` to improve performance.
/home/hmf6av/.local/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

[W 2025-06-23 16:15:29,293] Trial 6 failed with parameters: {'windows_batch_size': 128, 'inference_windows_batch_size': 85} because of the following error: OutOfMemoryError('CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacity of 31.73 GiB of which 127.69 MiB is free. Including non-PyTorch memory, this process has 31.39 GiB memory in use. Of the allocated memory 30.84 GiB is allocated by PyTorch, and 173.57 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)').
Traceback (most recent call last):
  File "/home/hmf6av/.local/lib/python3.11/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/sfs/gpfs/tardis/home/hmf6av/TimeSeriesComparisons/Time

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacity of 31.73 GiB of which 127.69 MiB is free. Including non-PyTorch memory, this process has 31.39 GiB memory in use. Of the allocated memory 30.84 GiB is allocated by PyTorch, and 173.57 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)